# Part 5 - Machine Learning

Finally we want to apply some machine learning to the taxi data. We will try to build a model for answering the following question:

    As a Taxi driver, what time and location would be best on a specific day to make most money.
    
This is a relevant question in order to help maximizing a Taxi drivers income. We will try to build a model which predicts the total fare amount for each date and hour and each (grid) location. This way the taxi driver can predict the expected total amount of money being made at a specific point in time at a specific location, so he can better plan his shifts.

Of course given the data that we currently have, we cannot precisely give an answer, because even locations with a low total fare amount may be attractive, as long as there are only very few taxi cabs. But we do not know how many taxi cabs were simply waiting or were driving around without passengers.

In [1]:
dwh_basedir = "/user/hadoop/nyc-dwh"
integrated_basedir = dwh_basedir + "/integrated"

# 0 Setup Environment

Before we begin, we create a local Spark session

## 0.1 Spark Session

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","64G") \
        .getOrCreate()

spark

## 0.2 Matplotlib

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# 1 Read Taxi Data

Now we can read in the taxi data from the structured zone.

In [4]:
hourly_taxi_trips = spark.read.parquet(integrated_basedir + "/taxi-trips-hourly")
hourly_taxi_trips.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,bank_holiday,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation
0,2013-04-12,14,3,6,288,543,9675.00,1012.03,11950.14,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
1,2013-04-12,14,2,2,238,399,3206.00,334.43,3765.53,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
2,2013-04-12,14,3,8,2,7,15.50,5.00,21.50,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
3,2013-04-12,14,-1,-1,838,1349,21554.16,1918.80,24907.67,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
4,2013-04-12,14,4,2,2900,5108,28534.00,2513.39,32720.89,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
5,2013-04-12,14,4,3,520,895,5617.00,517.76,6464.05,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
6,2013-04-12,14,2,3,29,33,829.50,37.25,907.99,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
7,2013-04-12,14,3,2,3322,5455,36235.50,3674.56,41991.23,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
8,2013-04-12,14,2,-1,3,3,30.00,3.00,34.50,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667
9,2013-04-12,14,2,7,4,4,145.00,10.00,162.33,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667


In [5]:
hourly_taxi_trips.printSchema()

root
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- lat_idx: integer (nullable = true)
 |-- long_idx: integer (nullable = true)
 |-- trip_count: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- holiday_description: string (nullable = true)
 |-- bank_holiday: boolean (nullable = true)
 |-- hourly_wind_speed: double (nullable = true)
 |-- hourly_temperature: double (nullable = true)
 |-- hourly_precipitation: double (nullable = true)
 |-- daily_temperature: double (nullable = true)
 |-- daily_wind_speed: double (nullable = true)
 |-- daily_precipitation: double (nullable = true)



# 2. Simple Model

## 2.1 Split Training and Validation set

As a first step, we split up the whole data set into a training and a validation data set. Typical data sets are split randomly, but for time series data sets a non-random split is preferrable in order to avoid an undesired information creep from future observations. Therefore we create a split filtering by date, such that about 80% of records are used for training and the remaining 20% of all records will be used for validation.

In [6]:
import datetime

training_fraction = 0.8
validation_fraction = 1 - training_fraction
split_date = datetime.date(2013, 1, 1) + datetime.timedelta(days=training_fraction*(365))
print("split_date=\"" + str(split_date) + "\"")

training_data = hourly_taxi_trips.filter(f.col("date") < split_date)
validation_data = hourly_taxi_trips.filter(f.col("date") >= split_date)

training_data_count = training_data.count()
validation_data_count = validation_data.count()

print("training_data count = " + str(training_data_count))
print("validation_data count = " + str(validation_data_count))

split_date="2013-10-20"
training_data count = 353211
validation_data count = 87593


## 2.2 Features

As a first step we need to create so called *features* from the training data set. Most PySpark ML algorithms expect two specific input columns: A so called *label* column containing the true value and a so called *features* column containing a vector of all variables used for prediction. 

The label column has to be a simple numeric value, in our case it will be the *total amount*.  The features column needs to contain the special data type *vector*, which is constructed from various attributes of our observations. Some of these attributes can be taken directly from our training data set, while other columns also need to be derived from the original values.

### Feature Engineering Building Blocks
PySpark provides lots of different feature engineering algorithms as building blocks. These building blocks are simple (or complex) transformations which typically will add new derived columns to a data frame. We will see how we can chain multiple of these components together into a so called *pipeline* later.

In [7]:
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml import Pipeline

### SQL Transformer
PySpark provides a very generic building block for transformation which simply executes some SQL. For example for creating a combined geo location inside a single column, we can use the following SQLTransformer

In [8]:
geo_location_transformer = SQLTransformer(
    statement="""
        SELECT
            *,
            CASE
                WHEN lat_idx IS NULL OR lat_idx < 0 THEN NULL
                WHEN long_idx IS NULL  OR long_idx < 0 THEN NULL
                ELSE concat(lat_idx, "/", long_idx) 
            END AS geo_location
        FROM __THIS__
    """
)

training_data_1 = geo_location_transformer.transform(training_data)
training_data_1.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,bank_holiday,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,geo_location
0,2013-04-12,14,3,6,288,543,9675.00,1012.03,11950.14,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,3/6
1,2013-04-12,14,2,2,238,399,3206.00,334.43,3765.53,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,2/2
2,2013-04-12,14,3,8,2,7,15.50,5.00,21.50,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,3/8
3,2013-04-12,14,-1,-1,838,1349,21554.16,1918.80,24907.67,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,None
4,2013-04-12,14,4,2,2900,5108,28534.00,2513.39,32720.89,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,4/2
5,2013-04-12,14,4,3,520,895,5617.00,517.76,6464.05,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,4/3
6,2013-04-12,14,2,3,29,33,829.50,37.25,907.99,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,2/3
7,2013-04-12,14,3,2,3322,5455,36235.50,3674.56,41991.23,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,3/2
8,2013-04-12,14,2,-1,3,3,30.00,3.00,34.50,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,None
9,2013-04-12,14,2,7,4,4,145.00,10.00,162.33,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,2/7


### One Hot Encoding
One important case where the original values cannot be used directly is categorial data. For example the geo location cannot be used as a numerical value. Therefore we need a transformation which creates numerical values from this categorial feature. PySpark provides the pair of a *string indexer* followed by *one hot encoding* to create a separate multidimensional vector for each categorial variable. The pattern is always the same:

```
categorial data => StringIndexer => OneHotEncoder => vector
```

Specifically the code for the geo location looks as follows:

In [9]:
# First create an index into all geo locations
geo_indexer = StringIndexer(
    inputCol="geo_location",
    outputCol="geo_location_idx",
    handleInvalid="keep"
)
geo_index_model = geo_indexer.fit(training_data_1)
training_data_2 = geo_index_model.transform(training_data_1)

# Now one-hot encode the generated index value
geo_encoder = OneHotEncoderEstimator(
    inputCols=["geo_location_idx"],
    outputCols=["geo_location_onehot"]
)
geo_encoder_model = geo_encoder.fit(training_data_2)
training_data_3 = geo_encoder_model.transform(training_data_2)

# Display some records
training_data_3.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,bank_holiday,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,geo_location,geo_location_idx,geo_location_onehot
0,2013-04-12,14,3,6,288,543,9675.00,1012.03,11950.14,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,3/6,22.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2013-04-12,14,2,2,238,399,3206.00,334.43,3765.53,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,2/2,13.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2013-04-12,14,3,8,2,7,15.50,5.00,21.50,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,3/8,32.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,2013-04-12,14,-1,-1,838,1349,21554.16,1918.80,24907.67,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,2013-04-12,14,4,2,2900,5108,28534.00,2513.39,32720.89,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,4/2,7.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
5,2013-04-12,14,4,3,520,895,5617.00,517.76,6464.05,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,4/3,4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
6,2013-04-12,14,2,3,29,33,829.50,37.25,907.99,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,2/3,17.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,2013-04-12,14,3,2,3322,5455,36235.50,3674.56,41991.23,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,3/2,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,2013-04-12,14,2,-1,3,3,30.00,3.00,34.50,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,2013-04-12,14,2,7,4,4,145.00,10.00,162.33,None,None,4.475,5.0,17.0,6.655347,4.985507,78.216667,2/7,31.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Vector Assembler

As already noted at the beginning of this section, PySpark requires all features to be available in a single column. This can be achieved by using a *vector assembler*, which will glue together all specified numerical and vector columns into a single vector column:

In [10]:
assembler = VectorAssembler(
    handleInvalid="skip",
    inputCols=[
        'hour',
        'bank_holiday',
        'geo_location_idx',
        'daily_temperature',
        'hourly_temperature',
        'daily_precipitation',
        'hourly_precipitation',
        'daily_wind_speed',
        'hourly_wind_speed'
    ],
    outputCol='features'
)

training_data_4 = assembler.transform(training_data_3)

training_data_4.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,geo_location,geo_location_idx,geo_location_onehot,features
0,2013-10-14,4,2,6,4,9,45.0,2.00,51.00,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,2/6,29.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 29.0, 14.502777777777775, 13.600000..."
1,2013-10-14,4,2,4,29,52,357.0,13.30,399.30,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,2/4,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 1.0, 14.502777777777775, 13.6000000..."
2,2013-10-14,4,2,2,15,16,215.5,22.03,262.69,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,2/2,13.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 13.0, 14.502777777777775, 13.600000..."
3,2013-10-14,4,2,5,26,43,416.5,5.60,464.09,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,2/5,19.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 19.0, 14.502777777777775, 13.600000..."
4,2013-10-14,4,4,3,51,76,829.0,93.62,1068.57,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,4/3,4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 4.0, 14.502777777777775, 13.6000000..."
5,2013-10-14,4,1,0,392,670,6251.0,536.69,7246.94,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,1/0,16.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 16.0, 14.502777777777775, 13.600000..."
6,2013-10-14,4,-1,4,9,14,122.5,11.50,143.00,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 50.0, 14.502777777777775, 13.600000..."
7,2013-10-14,4,0,2,55,86,687.5,53.09,795.59,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,0/2,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 10.0, 14.502777777777775, 13.600000..."
8,2013-10-14,4,3,3,9,15,140.0,5.20,159.53,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,3/3,15.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 15.0, 14.502777777777775, 13.600000..."
9,2013-10-14,4,1,7,4,4,67.0,10.00,81.00,Columbus Day,...,0.75,13.6,0.0,14.502778,1.532609,0.0,1/7,23.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 1.0, 23.0, 14.502777777777775, 13.600000..."


### Pipeline

Now we have met all relevant building blocks. Instead of manually chaining these transformations together, you always should use a *pipeline*, where you can simply specify all transformations to apply. The pipeline also takes care of performing any `fit` phase of some transformers (like `StringIndexer` or `OneHotEncoderEstimator`).

In [11]:
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml import Pipeline

feature_pipeline = Pipeline(
    stages = [
        SQLTransformer(
            statement="""
                SELECT
                    total_amount,
                    date,
                    hour,
                    daily_temperature,
                    hourly_temperature,
                    daily_precipitation,
                    hourly_precipitation,
                    daily_wind_speed,
                    hourly_wind_speed,
                    month(`date`) - 1 AS `month_idx`,
                    dayofweek(`date`) - 1 AS `weekday_idx`,
                    CASE
                        WHEN lat_idx IS NULL OR lat_idx < 0 THEN NULL
                        WHEN long_idx IS NULL  OR long_idx < 0 THEN NULL
                        ELSE concat(lat_idx, "/", long_idx) 
                    END AS geo_location,
                    CASE WHEN
                        bank_holiday = true THEN 1
                        ELSE 0
                    END AS bank_holiday
                FROM __THIS__
            """
        ),
        StringIndexer(
            inputCol="geo_location",
            outputCol="geo_location_idx",
            handleInvalid="keep"
        ),
        OneHotEncoderEstimator(
            inputCols=["geo_location_idx"],
            outputCols=["geo_location_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["hour"],
            outputCols=["hour_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["month_idx"],
            outputCols=["month_onehot"],
            handleInvalid="keep"
        ),
        OneHotEncoderEstimator(
            inputCols=["weekday_idx"],
            outputCols=["weekday_onehot"]
        ),
        
        VectorAssembler(
            handleInvalid="skip",
            inputCols=[
                'weekday_onehot',
                'hour_onehot',
                'bank_holiday',
                'geo_location_idx',
                'daily_temperature',
                'hourly_temperature',
                'daily_precipitation',
                'hourly_precipitation',
                'daily_wind_speed',
                'hourly_wind_speed'
            ],
            outputCol='features'
        )
    ]
)

feature_model = feature_pipeline.fit(training_data)

The feature model now contains all stages and can be used as a transformer. Let's transform and display the training data as a quick test.

In [12]:
features_training_data = feature_model.transform(training_data)
features_training_data.limit(10).toPandas()

,total_amount,date,hour,daily_temperature,hourly_temperature,daily_precipitation,hourly_precipitation,daily_wind_speed,hourly_wind_speed,month_idx,weekday_idx,geo_location,bank_holiday,geo_location_idx,geo_location_onehot,hour_onehot,month_onehot,weekday_onehot,features
0,11950.14,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,3/6,0,22.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,3765.53,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,2/2,0,13.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,21.50,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,3/8,0,32.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,24907.67,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,None,0,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,32720.89,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,4/2,0,7.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
5,6464.05,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,4/3,0,4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
6,907.99,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,2/3,0,17.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
7,41991.23,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,3/2,0,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
8,34.50,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,None,0,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
9,162.33,2013-04-12,14,6.655347,5.0,78.216667,17.0,4.985507,4.475,3,5,2/7,0,31.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


Since we specified `skip` as the strategy to handle invalid records in the vector assembler, let us check how many records where dropped from the training data.

In [13]:
features_data_count = feature_model.transform(training_data).count()

print("feature_data_count = " + str(features_data_count))
print("training_data_count = " + str(training_data_count))
print("skipped records = " + str(training_data_count - features_data_count))

feature_data_count = 332985
training_data_count = 353211
skipped records = 20226


## 2.3 Model

So far we have only prepared the data by adding a feature column containg some information which should be used as independant variables for prediction. Now we finally want to train a model which makes use of these features and predict the total amount for every date, hour and geo location.

We create another pipeline, which contains the feature pipeline as its first entry and a simple linear regression as its second stage.

In [14]:
pred_pipeline = Pipeline(stages=[
    feature_pipeline,
    LinearRegression(
        featuresCol="features",
        labelCol="total_amount",
        predictionCol="pred_total_amount"
    )
])

pred_model = pred_pipeline.fit(training_data)

## 2.4 Prediction

The `pred_model` now contains all feature transformation steps of the feature pipeline and a linear model. We can directly use this model for performing predictions of the total amount.

Now we use the validation data for prediction, which we set aside at the beginning and which could not influence the training phase in any way.

In [15]:
pred_validation_data = pred_model.transform(validation_data)

We now want to display some columns of the predicted values

In [16]:
pred_validation_data\
    .orderBy("date", "hour") \
    .select(
        "date", "hour",
        "geo_location",
        "total_amount",
        "pred_total_amount"
    ) \
    .limit(10)\
    .toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-10-20,1,0/2,11342.92,8197.141573
1,2013-10-20,1,1/5,62.29,1771.350731
2,2013-10-20,1,None,393.87,-2084.123774
3,2013-10-20,1,2/5,437.02,5883.856870
4,2013-10-20,1,0/8,16.50,3570.572167
5,2013-10-20,1,2/1,66397.38,9482.299741
6,2013-10-20,1,2/8,214.00,486.192563
7,2013-10-20,1,None,4334.17,-2084.123774
8,2013-10-20,1,None,26.90,-2084.123774
9,2013-10-20,1,2/6,62.49,3313.540533


## 2.5 Validation

When looking at the predictions and comparing them with the true total amounts, we already suspect that our model does not perform very well. Now we want to quantify the goodness of fit using an appropriate evaluator. Note that evaluation should always be performed with the validation data, since we are not interested very much into the question how well a model describes the training data, but instead we want to understand how well the model performs with new data, which was not used during the training phase.

In [17]:
from pyspark.ml.evaluation import *

evaluator = RegressionEvaluator(
    labelCol = "total_amount",
    predictionCol = "pred_total_amount",
    metricName = "rmse"
)

rmse = evaluator.evaluate(pred_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 11044.40992114027
real_avg = 5866.227672759218


Apparently our model is not very good, since its RMSE is about twice as large as the average value.

## 2.6 Baseline Model

We already saw that the model is not very good. But what can we expect? It is always helpful to come up with a very simple base line model, and every true model should better beat the base line model. In our case, we simply use the average total amount as a constant base line model.

In [18]:
avg_total_amount = training_data.select(f.avg("total_amount")).first()[0]
baseline_validation_data = validation_data.withColumn("pred_total_amount", f.lit(avg_total_amount))

Using the base line model, we can now calculate the RMSE of this model as a baseline.

In [19]:
rmse = evaluator.evaluate(baseline_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 12121.134683259721
real_avg = 5866.227672759217


# 3. Improve Model

## 3.1 Integrate information from the past

Depending on the scenario, it can be completely legal to use data from the past as an additional feature. In this example, we assume that we can use the total amount from at least three days ago for the same location as an additional features. This implies that we assume that these numbers ares available at the time when new predictions are made.

In other scenarios, the minimum amount of time to go back into the past, may be much larger or smaller. The importat question here is always what data is available when a new prediciton is performed.

One important aspect of this approach is that no data may be available for some locations. But since our algorithms always require that all observations contain valid numbers, we fill the overall average of the metric over all locations for a specific date and hour.

### Using Window Functions

By using so called window functions, we can access data from different rows for the same geo location. In this example, we calculate the average of the total amount of the same hour, but seven days ago. We will add more data from the past afterwards.

A Spark window function can be used with any aggregation by specifying the range in a `over` clause

```
f.avg("total_amount") \
    .over(
        Window.partitionBy("lat_idx", "long_idx") \
            .orderBy("ts") \
            .rangeBetween(-7*24*60*60,-(7*24-1)*60*60)
    )
```

In [20]:
from pyspark.sql import Window

hourly_taxi_trips_ext = hourly_taxi_trips \
    .withColumn("ts", f.unix_timestamp(f.concat(f.col("date"),f.lit(" "),f.col("hour")), "yyyy-MM-dd H")) \
    .withColumn("total_amount_P7D",
        f.avg("total_amount").over(Window.partitionBy("lat_idx", "long_idx").orderBy("ts").rangeBetween(-7*24*60*60,-(7*24-1)*60*60))
    )

In [21]:
hourly_taxi_trips_ext.filter(f.col("date") > "2013-02-01").limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,bank_holiday,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,ts,total_amount_P7D
0,2013-02-02,0,1,0,3084,5691,35249.0,4251.12,42693.57,None,None,NaN,-3.3,0.0,-4.370833,3.261905,0.0,1359763200,35454.620
1,2013-02-02,1,1,0,3072,5814,36131.5,4534.85,43823.40,None,None,2.1,-3.3,0.0,-4.370833,3.261905,0.0,1359766800,33913.870
2,2013-02-02,2,1,0,2930,5406,34157.5,4162.83,41329.23,None,None,3.6,-3.9,0.0,-4.370833,3.261905,0.0,1359770400,28353.440
3,2013-02-02,3,1,0,2013,3709,25022.5,2787.28,29859.68,None,None,6.2,-4.4,0.0,-4.370833,3.261905,0.0,1359774000,19603.440
4,2013-02-02,4,1,0,1004,1795,13438.0,1268.93,15796.78,None,None,4.1,-4.4,0.0,-4.370833,3.261905,0.0,1359777600,9933.800
5,2013-02-02,5,1,0,348,615,5087.5,448.56,5932.96,None,None,5.7,-4.4,0.0,-4.370833,3.261905,0.0,1359781200,4709.040
6,2013-02-02,6,1,0,248,382,3529.0,325.21,4027.36,None,None,4.6,-5.0,0.0,-4.370833,3.261905,0.0,1359784800,4422.805
7,2013-02-02,7,1,0,323,510,4486.0,556.77,5278.87,None,None,3.1,-5.6,0.0,-4.370833,3.261905,0.0,1359788400,6181.335
8,2013-02-02,8,1,0,646,1068,6738.0,734.98,7871.93,None,None,1.5,-6.1,0.0,-4.370833,3.261905,0.0,1359792000,9090.230
9,2013-02-02,9,1,0,984,1634,9785.5,1127.77,11481.32,None,None,3.6,-6.7,0.0,-4.370833,3.261905,0.0,1359795600,11985.220


### Add more history

Now we add more columns with more hours from the past by using multiple window expressions. We will denote the columns with `total_amount_<ISO Duration>`.

In [22]:
from pyspark.sql import Window

# Helper function for creating a history column name
def column_name(prefix, days, hours):
    if hours > 0:
        timedelta = "P" + str(days) + "DT" + str(hours) + "H"
    else:
        timedelta = "P" + str(days) + "D"
        
    return prefix + "_" + timedelta


# Function which adds history information for "total_amount" going back the specified number of days and hours
def add_history(df, days, hours):
    column = column_name("total_amount", days, hours)
    window_start = (days*24+hours)*60*60
    window_end = window_start - 60*60
    result = df.withColumn(column,
                f.avg("total_amount").over(Window.partitionBy("lat_idx", "long_idx").orderBy("ts").rangeBetween(-window_start, -window_end))
            ) 
    return result

# Add timestamp column, which is required for the Window function
hourly_taxi_trips_ext = hourly_taxi_trips \
    .withColumn("ts", f.unix_timestamp(f.concat(f.col("date"),f.lit(" "),f.col("hour")), "yyyy-MM-dd H"))

# Add history for each hour for seven days
df = hourly_taxi_trips_ext
for day in range(10,2,-1):
    for hour in range(23,-1,-1):
        df = add_history(df, day, hour)

In [23]:
df.filter(f.col("date") > "2013-02-01").limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,total_amount_P3DT9H,total_amount_P3DT8H,total_amount_P3DT7H,total_amount_P3DT6H,total_amount_P3DT5H,total_amount_P3DT4H,total_amount_P3DT3H,total_amount_P3DT2H,total_amount_P3DT1H,total_amount_P3D
0,2013-02-02,0,1,0,3084,5691,35249.0,4251.12,42693.57,None,...,15601.280,16972.870,24001.425,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490
1,2013-02-02,1,1,0,3072,5814,36131.5,4534.85,43823.40,None,...,16972.870,24001.425,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730
2,2013-02-02,2,1,0,2930,5406,34157.5,4162.83,41329.23,None,...,24001.425,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380
3,2013-02-02,3,1,0,2013,3709,25022.5,2787.28,29859.68,None,...,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735
4,2013-02-02,4,1,0,1004,1795,13438.0,1268.93,15796.78,None,...,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995
5,2013-02-02,5,1,0,348,615,5087.5,448.56,5932.96,None,...,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900
6,2013-02-02,6,1,0,248,382,3529.0,325.21,4027.36,None,...,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995
7,2013-02-02,7,1,0,323,510,4486.0,556.77,5278.87,None,...,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995,20798.740
8,2013-02-02,8,1,0,646,1068,6738.0,734.98,7871.93,None,...,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995,20798.740,23241.515
9,2013-02-02,9,1,0,984,1634,9785.5,1127.77,11481.32,None,...,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995,20798.740,23241.515,20071.160


### Impute Missing Values

We now have the problem that we do not neccessarily have a complete history for every geo location. In this case, the data set will contain NULL values which will cause errors in the algorithms later on. Therefore we need to impute some values. Spark provides some functionality for doing that, but that is very coarse (you can only replace all NULL values with a single common value). Instead we want to replace each NULL value by an average of every specific geo location.

In [24]:
# First calculate average per geo location
taxi_trips_avg = hourly_taxi_trips \
    .groupBy("lat_idx", "long_idx") \
    .agg(
        f.avg("fare_amount").alias("avg_fare_amount"),
        f.avg("tip_amount").alias("avg_tip_amount"),
        f.avg("total_amount").alias("avg_total_amount")
    )

# Helper function for imputing missing values for a specific hour on a specific date
def impute_history(df, days, hours):
    column = column_name("total_amount", days, hours)
    result =  df.withColumn(column, f.coalesce(f.col(column), f.col("avg_total_amount")))
    return result

df = df.join(taxi_trips_avg, ["lat_idx","long_idx"], how="leftOuter")
for day in range(10,2,-1):
    for hour in range(23,-1,-1):
        df = impute_history(df, day, hour)
        
# Clean up DataFrame        
df = df.drop("avg_fare_amount","avg_tip_amount","avg_total_amount")

# Finally cache the result
hourly_taxi_trips_ext = df.cache()

Let us display some records again

In [25]:
hourly_taxi_trips_ext.filter(f.col("date") > "2013-02-01").limit(10).toPandas()

,lat_idx,long_idx,date,hour,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,total_amount_P3DT9H,total_amount_P3DT8H,total_amount_P3DT7H,total_amount_P3DT6H,total_amount_P3DT5H,total_amount_P3DT4H,total_amount_P3DT3H,total_amount_P3DT2H,total_amount_P3DT1H,total_amount_P3D
0,1,0,2013-02-02,0,3084,5691,35249.0,4251.12,42693.57,None,...,15601.280,16972.870,24001.425,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490
1,1,0,2013-02-02,1,3072,5814,36131.5,4534.85,43823.40,None,...,16972.870,24001.425,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730
2,1,0,2013-02-02,2,2930,5406,34157.5,4162.83,41329.23,None,...,24001.425,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380
3,1,0,2013-02-02,3,2013,3709,25022.5,2787.28,29859.68,None,...,29154.720,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735
4,1,0,2013-02-02,4,1004,1795,13438.0,1268.93,15796.78,None,...,30482.215,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995
5,1,0,2013-02-02,5,348,615,5087.5,448.56,5932.96,None,...,32745.990,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900
6,1,0,2013-02-02,6,248,382,3529.0,325.21,4027.36,None,...,34820.995,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995
7,1,0,2013-02-02,7,323,510,4486.0,556.77,5278.87,None,...,30702.955,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995,20798.740
8,1,0,2013-02-02,8,646,1068,6738.0,734.98,7871.93,None,...,21801.965,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995,20798.740,23241.515
9,1,0,2013-02-02,9,984,1634,9785.5,1127.77,11481.32,None,...,14469.490,9521.730,6108.380,4940.735,4026.995,6018.900,13335.995,20798.740,23241.515,20071.160


## 3.2 Split Training and Validation set

As a first step, we split up the whole data set into a training and a validation data set. Typical data sets are split randomly, but for time series data sets a non-random split is preferrable in order to avoid an undesired information creep from future observations. Therefore we create a split filtering by date, such that about 80% of records are used for training and the remaining 20% of all records will be used for validation.

In [26]:
import datetime

training_fraction = 0.8
validation_fraction = 1 - training_fraction
split_date = datetime.date(2013, 1, 7) + datetime.timedelta(days=training_fraction*(365-7))
print("split_date=\"" + str(split_date) + "\"")

training_data = hourly_taxi_trips_ext.filter(f.col("date") < split_date)
validation_data = hourly_taxi_trips_ext.filter(f.col("date") >= split_date)

training_data_count = training_data.count()
validation_data_count = validation_data.count()

print("training_data count = " + str(training_data_count))
print("validation_data count = " + str(validation_data_count))

split_date="2013-10-20"
training_data count = 353211
validation_data count = 87593


## 3.3 Create Features and Train Model

Using building blocks of the PySpark ML package, we create a machine learning pipeline with all feature engineering steps and the regression.

### Bucketing

For some numerical features (like temperature and wind speed), it may be more appropriate to model them as categorical features. This can be done by *bucketing* as follows:

In [27]:
# Create Buckets
bucketizer = Bucketizer(
    inputCol="daily_temperature",
    outputCol="daily_temperature_bucket",
    handleInvalid="keep",
    splits=[-float("inf"),-10,0,10,15,20,25,30,float("inf")]
)
training_data_1 = bucketizer.transform(training_data)

# One Hot encode buckets
encoder = OneHotEncoderEstimator(
    inputCols=["daily_temperature_bucket"],
    outputCols=["daily_temperature_onehot"]
)
encoder_model = encoder.fit(training_data_1)
training_data_2 = encoder_model.transform(training_data_1)

In [28]:
training_data_2.limit(10).toPandas()

,lat_idx,long_idx,date,hour,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,total_amount_P3DT7H,total_amount_P3DT6H,total_amount_P3DT5H,total_amount_P3DT4H,total_amount_P3DT3H,total_amount_P3DT2H,total_amount_P3DT1H,total_amount_P3D,daily_temperature_bucket,daily_temperature_onehot
0,1,0,2013-01-01,0,1971,3748,24045.28,2280.49,28378.72,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
1,1,0,2013-01-01,1,1663,3103,21061.00,2064.42,24880.12,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
2,1,0,2013-01-01,2,1448,2822,18990.00,1752.44,22257.69,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
3,1,0,2013-01-01,3,1384,2699,18313.50,1807.10,21576.46,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
4,1,0,2013-01-01,4,1116,2116,15065.00,1242.32,17490.56,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
5,1,0,2013-01-01,5,707,1323,10236.00,724.91,11721.61,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
6,1,0,2013-01-01,6,426,792,6850.50,532.23,7640.73,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
7,1,0,2013-01-01,7,333,611,5112.50,360.79,5682.14,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
8,1,0,2013-01-01,8,237,415,3495.50,266.77,3924.12,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
9,1,0,2013-01-01,9,301,538,4103.50,326.64,4629.59,New Year Day,...,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,19074.220188,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"


### Pipeline

Now we can create a more extensive pipeline, which makes use of more features and which also performs bucketing of the weather data.

In particulat the pipeline performs the following steps:
* one hot encode geo location
* one hot encode hour
* one hot encode day of week
* bucketize all weather measurements
* perform regression
* truncate predictions to zero from below

In [29]:
total_amount_history = [column_name("total_amount", day, hour) for day in range(10,2,-1) for hour in range(23,-1,-1)]
total_amount_history[:5]

['total_amount_P10DT23H',
 'total_amount_P10DT22H',
 'total_amount_P10DT21H',
 'total_amount_P10DT20H',
 'total_amount_P10DT19H']

In [30]:
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml import Pipeline

pipeline = Pipeline(
    stages = [
        SQLTransformer(
            statement="""
                SELECT
                    *,
                    CASE WHEN
                        bank_holiday = true THEN 0
                        ELSE dayofweek(`date`)
                    END AS weekday_idx,
                    CASE
                        WHEN lat_idx IS NULL OR lat_idx < 0 THEN NULL
                        WHEN long_idx IS NULL  OR long_idx < 0 THEN NULL
                        ELSE concat(lat_idx, "/", long_idx) 
                    END AS geo_location
                FROM __THIS__
            """
        ),
        StringIndexer(
            inputCol="geo_location",
            outputCol="geo_location_idx",
            handleInvalid="keep"
        ),
        OneHotEncoderEstimator(
            inputCols=["geo_location_idx"],
            outputCols=["geo_location_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["hour"],
            outputCols=["hour_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["weekday_idx"],
            outputCols=["weekday_onehot"]
        ),
        
        Bucketizer(
            inputCol="daily_temperature",
            outputCol="daily_temperature_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),-10,0,10,15,20,25,30,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["daily_temperature_bucket"],
            outputCols=["daily_temperature_onehot"]
        ),
        Bucketizer(
            inputCol="hourly_temperature",
            outputCol="hourly_temperature_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),-10,0,10,15,20,25,30,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["hourly_temperature_bucket"],
            outputCols=["hourly_temperature_onehot"]
        ),
        Bucketizer(
            inputCol="daily_precipitation",
            outputCol="daily_precipitation_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,100,200,300,400,500,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["daily_precipitation_bucket"],
            outputCols=["daily_precipitation_onehot"]
        ),
        Bucketizer(
            inputCol="hourly_precipitation",
            outputCol="hourly_precipitation_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,50,100,150,200,250,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["hourly_precipitation_bucket"],
            outputCols=["hourly_precipitation_onehot"]
        ),
        Bucketizer(
            inputCol="daily_wind_speed",
            outputCol="daily_wind_speed_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,1,2,3,4,5,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["daily_wind_speed_bucket"],
            outputCols=["daily_wind_speed_onehot"]
        ),
        Bucketizer(
            inputCol="hourly_wind_speed",
            outputCol="hourly_wind_speed_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,1,2,3,4,5,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["hourly_wind_speed_bucket"],
            outputCols=["hourly_wind_speed_onehot"]
        ),
        
        # Linear Prediction
        VectorAssembler(
            handleInvalid="skip",
            inputCols=total_amount_history + [
                'weekday_onehot',
                'hour_onehot',
                'geo_location_onehot',
                'daily_temperature_onehot',
                'hourly_temperature_onehot',
                'daily_precipitation_onehot',
                'hourly_precipitation_onehot',
                'daily_wind_speed_onehot',
                'hourly_wind_speed_onehot'
            ],
            outputCol='features'
        ),
        LinearRegression(
            featuresCol="features",
            labelCol="total_amount",
            predictionCol="pred_total_amount"
        ),
        SQLTransformer(
            statement="""
                SELECT
                    date,
                    hour,
                    geo_location,
                    total_amount,
                    CASE 
                        WHEN pred_total_amount > 0 THEN pred_total_amount
                        ELSE 0
                    END AS pred_total_amount
                FROM __THIS__
            """
        )
    ]
)

pred_model = pipeline.fit(training_data)

Again let us check how many training records have been dropped during the pipeline.

In [31]:
features_data_count = pred_model.transform(training_data).count()

print("training_data_count = " + str(training_data_count))
print("feature_data_count = " + str(features_data_count))
print("skipped records = " + str(training_data_count - features_data_count))

training_data_count = 353211
feature_data_count = 332985
skipped records = 20226


## 3.4 Prediction

In [32]:
pred_validation_data = pred_model.transform(validation_data)
pred_validation_data\
    .orderBy("date", "hour") \
    .select(
        "date", "hour",
        "geo_location",
        "total_amount",
        "pred_total_amount"
    ) \
    .limit(10)\
    .toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-10-20,1,1/4,8.50,0.000000
1,2013-10-20,1,0/5,33.50,0.000000
2,2013-10-20,1,0/0,12449.44,10828.902537
3,2013-10-20,1,2/2,2547.50,2765.638968
4,2013-10-20,1,None,64.50,318.602691
5,2013-10-20,1,1/0,39846.24,36933.379346
6,2013-10-20,1,4/0,13.50,0.000000
7,2013-10-20,1,1/7,188.33,64.344997
8,2013-10-20,1,3/1,50100.46,48461.728746
9,2013-10-20,1,None,26.90,380.955587


## 3.6 Validation

In [33]:
from pyspark.ml.evaluation import *

evaluator = RegressionEvaluator(
    labelCol = "total_amount",
    predictionCol = "pred_total_amount",
    metricName = "rmse"
)

rmse = evaluator.evaluate(pred_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 3013.7773375483453
real_avg = 5866.227672759241


## 3.7 Baseline Model

In order to make sense of the number, we use a simple base line model as comparison again. This time we simply predict the total amount by using the previous total amount from seven days ago.

In [34]:
baseline_validation_data = validation_data.withColumn("pred_total_amount", f.col("total_amount_P7D"))

rmse = evaluator.evaluate(baseline_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 3446.4030875587664
real_avg = 5866.22767275924


# 4. Best Time and Location

Although the predicted values are not really satisfying so far, they are good enough for deciding when to make most money. In order to underline this claim, let us compare the top ten hours and locations.

In [35]:
pred_validation_data.filter("date='2013-11-01'") \
    .select("date", "hour", "geo_location", "total_amount", "pred_total_amount") \
    .orderBy(f.desc("total_amount")) \
    .limit(10).toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-11-01,20,2/1,83920.03,76460.136535
1,2013-11-01,22,2/1,77756.42,79642.917695
2,2013-11-01,21,2/1,76480.11,79331.697863
3,2013-11-01,19,2/1,74639.26,71792.328033
4,2013-11-01,1,1/1,73207.54,35209.038901
5,2013-11-01,0,2/1,66320.98,51407.497513
6,2013-11-01,22,3/1,65089.26,68471.658185
7,2013-11-01,20,3/1,64987.55,64523.583869
8,2013-11-01,0,1/1,64479.22,44607.915631
9,2013-11-01,18,2/1,64298.62,63293.174038


In [36]:
pred_validation_data.filter("date='2013-11-01'") \
    .select("date", "hour", "geo_location", "total_amount", "pred_total_amount") \
    .orderBy(f.desc("pred_total_amount")) \
    .limit(10).toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-11-01,22,2/1,77756.42,79642.917695
1,2013-11-01,21,2/1,76480.11,79331.697863
2,2013-11-01,20,2/1,83920.03,76460.136535
3,2013-11-01,19,2/1,74639.26,71792.328033
4,2013-11-01,19,3/1,62415.96,69131.322740
5,2013-11-01,22,3/1,65089.26,68471.658185
6,2013-11-01,12,3/1,60247.81,65859.630698
7,2013-11-01,14,3/1,63646.01,65496.343209
8,2013-11-01,11,3/1,55791.27,64616.856095
9,2013-11-01,20,3/1,64987.55,64523.583869


## 4.1 Top 10 Recommendations

Let us find out how good the recommendatations of our algorithm would be. We pick the ten best hour-locations for each day from the real data, the predicted data and the baseline model. For each selection, we also compute the *real total revenue* for all these location-hours.

### Real 10 best location-hours

In [37]:
from pyspark.sql import Window

real_best_locations = pred_validation_data \
    .select(
        f.col("date"),
        f.col("hour"),
        f.col("total_amount"),
        f.col("geo_location"),
        f.row_number().over(Window.partitionBy("date").orderBy(f.col("total_amount").desc())).alias("row_number")
    ) \
    .filter(f.col("row_number") < 10)

real_best_locations.limit(10).toPandas()

,date,hour,total_amount,geo_location,row_number
0,2013-11-18,18,76061.25,3/1,1
1,2013-11-18,19,74432.77,3/1,2
2,2013-11-18,20,71042.44,3/1,3
3,2013-11-18,19,70469.41,2/1,4
4,2013-11-18,18,70008.61,2/1,5
5,2013-11-18,21,67143.80,3/1,6
6,2013-11-18,20,63513.76,2/1,7
7,2013-11-18,22,59311.04,3/1,8
8,2013-11-18,21,57512.11,2/1,9
9,2013-11-23,1,82601.18,1/1,1


In [38]:
real_top10_totals = real_best_locations.select(
    f.sum(f.col("total_amount"))
).first()[0]

print("real_top10_totals = " + str(real_top10_totals))

real_top10_totals = 45671779.01


### Predicted 10 best location-hours

In [39]:
pred_best_locations = pred_validation_data \
    .select(
        f.col("date"),
        f.col("hour"),
        f.col("total_amount"),
        f.col("geo_location"),
        f.row_number().over(Window.partitionBy("date").orderBy(f.col("pred_total_amount").desc())).alias("row_number")
    ) \
    .filter(f.col("row_number") < 10)

pred_best_locations.limit(10).toPandas()

,date,hour,total_amount,geo_location,row_number
0,2013-11-18,19,74432.77,3/1,1
1,2013-11-18,19,70469.41,2/1,2
2,2013-11-18,20,63513.76,2/1,3
3,2013-11-18,20,71042.44,3/1,4
4,2013-11-18,18,70008.61,2/1,5
5,2013-11-18,18,76061.25,3/1,6
6,2013-11-18,21,67143.80,3/1,7
7,2013-11-18,21,57512.11,2/1,8
8,2013-11-18,14,56426.48,3/1,9
9,2013-11-23,1,82601.18,1/1,1


In [40]:
pred_top10_totals = pred_best_locations.select(
    f.sum(f.col("total_amount"))
).first()[0] 

print("pred_top10_totals = " + str(pred_top10_totals))

pred_top10_totals = 43555425.300000004


### Baseline 10 best location-hours

In [41]:
baseline_best_locations = validation_data \
    .select(
        f.col("date"),
        f.col("hour"),
        f.col("total_amount"),
        f.concat(f.col("lat_idx"), f.lit("/"), f.col("long_idx")).alias("geo_location"),
        f.row_number().over(Window.partitionBy("date").orderBy(f.col("total_amount_P7D").desc())).alias("row_number")
    ) \
    .filter(f.col("row_number") < 10)

baseline_best_locations.limit(10).toPandas()

,date,hour,total_amount,geo_location,row_number
0,2013-11-18,19,74432.77,3/1,1
1,2013-11-18,18,76061.25,3/1,2
2,2013-11-18,18,70008.61,2/1,3
3,2013-11-18,20,71042.44,3/1,4
4,2013-11-18,19,70469.41,2/1,5
5,2013-11-18,21,67143.80,3/1,6
6,2013-11-18,20,63513.76,2/1,7
7,2013-11-18,17,55327.07,2/1,8
8,2013-11-18,17,55558.54,3/1,9
9,2013-11-23,1,82601.18,1/1,1


In [42]:
baseline_top10_totals = baseline_best_locations.select(
    f.sum(f.col("total_amount"))
).first()[0] 

print("baseline_top10_totals = " + str(baseline_top10_totals))

baseline_top10_totals = 43536933.27
